# Odporúčanie

In [25]:
%load_ext autoreload
%autoreload all

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
# dependencies
import pandas as pd
import wandb
from dotenv import load_dotenv

from _helpers import constants
from _helpers.verify_submission.verify_subm import main as verify_subm
from _helpers.score_submission.score_subm import main as score_subm
from models.model_random import ModelRandom
from models.model_nochange import ModelNoChange

In [27]:
# Load env variables from .env file
load_dotenv()

False

Nainicializujeme Weights and Biases (logovací nástroj pre strojové učenie)

In [28]:
wandb.login()

True

In [29]:
df_train = pd.read_csv(constants.TRAIN)

Spustíme trénovanie

In [30]:
models = {
    'random': ModelRandom(),
    'nochange': ModelNoChange(),
}

In [31]:
run = 1
notes = ''
params = {
    'model': 'nochange',
}

wandb_run = wandb.init(entity='mcfreddie777', project="dp-recsys", name=f'model_{params["model"]}_run_{run}', notes=notes)
wandb_run.config.update(params)

model = models[params['model']]
model.update(params)
model.fit(df_train)

fatal: stat 'src/prediction.ipynb': No such file or directory


Predikujeme odporúčania

In [32]:
df_test = pd.read_csv(constants.TEST)
df_recommendations = model.predict(df_test)

Verifikujeme predikcie

In [33]:
verify_subm(df_subm=df_recommendations,df_test=df_test)

Checking for required columns in the submission file...
> check passed
Checking for duplicate sessions in the submission file...
> check passed
Checking that all the required sessions are present in submission...
> check passed
All checks passed


In [34]:
df_recommendations.to_csv(constants.OUTPUT_DIR / f'submission_popular_{params["model"]}_{run}.csv', index=False)

Vypočítame si MRR na dátach

In [35]:
df_gt = pd.read_csv(constants.GROUND_TRUTH)
mrr,map3 = score_subm(df_subm=df_recommendations,df_gt=df_gt)
wandb_run.log({"mrr":mrr, "map3":map3})

Mean reciprocal rank:      0.2242
Mean average precision @3: 0.0668


In [36]:
wandb_run.finish()


map3,▁
mrr,▁
map3,0.0668
mrr,0.2242
